In [1]:
import tensorflow as tf
from collections import namedtuple
HParams = namedtuple('HParams',
                     'mode, min_lr, lr, batch_size, '
                     'enc_layers, enc_timesteps, dec_timesteps, '
                     'min_input_len, num_hidden, emb_dim, max_grad_norm, '
                     'num_softmax_samples')
hps = HParams(
      mode="train",  # train, eval, decode
      min_lr=0.01,  # min learning rate.
      lr=0.15,  # learning rate
      batch_size=4,
      enc_layers=4,
      enc_timesteps=120,
      dec_timesteps=30,
      min_input_len=2,  # discard articles/summaries < than this
      num_hidden=200,  # for rnn cell
      emb_dim=128,  # If 0, don't use embedding
      max_grad_norm=2,
      num_softmax_samples=4096)  # If 0, no sampled softmax.
_articles = tf.placeholder(tf.int32,
                                    [hps.batch_size, hps.enc_timesteps],
                                    name='articles')
encoder_inputs = tf.unstack(tf.transpose(_articles))
vsize=4000
embedding = tf.get_variable(
            'embedding', [vsize, hps.emb_dim], dtype=tf.float32,
            initializer=tf.truncated_normal_initializer(stddev=1e-4))

emb_encoder_inputs = [tf.nn.embedding_lookup(embedding, x)
                              for x in encoder_inputs]
article_lens = tf.placeholder(tf.int32, [hps.batch_size],
                                        name='article_lens')
print(emb_encoder_inputs[0].get_shape().as_list())
num_hidden=200
for layer_i in range(hps.enc_layers):
    with tf.variable_scope('encoder%d' % layer_i):
        cell_fw = tf.nn.rnn_cell.LSTMCell(
            num_hidden,
            initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=123),
            state_is_tuple=True)
        cell_bw = tf.nn.rnn_cell.LSTMCell(
            num_hidden,
            initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=113),
            state_is_tuple=True)
        # emb_encoder_inputs的shape是[batch_size,2*num_hidden]
        (emb_encoder_inputs, fw_state, _) = tf.nn.static_bidirectional_rnn(
            cell_fw, cell_bw, emb_encoder_inputs, dtype=tf.float32,
            sequence_length=article_lens)
print(emb_encoder_inputs[0].get_shape().as_list())
encoder_outputs = emb_encoder_inputs
encoder_outputs = [tf.reshape(x, [hps.batch_size, 1, 2*hps.num_hidden]) for x in encoder_outputs]
print(encoder_outputs[0].get_shape().as_list())
enc_top_states = tf.concat(axis=1, values=encoder_outputs)
print(enc_top_states.get_shape().as_list())


[4, 128]


[4, 1, 400]
[4, 120, 400]
